In [ ]:
#Conda 환경 설정
!conda create -n convert
!conda activate convert
!conda install jupyter notebook
!python3 -m ipykernel install --user --name convert --display-name "convert"

In [ ]:
#update
!sudo apt update --yes
!sudo apt upgrade --yes
!sudo apt autoremove --yes

In [ ]:
#lib 설치
!conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch
!conda install -c conda-forge onnx

In [ ]:
%cd /home/ai/Desktop/work/ai_boostcamp/final/pth_convert

Pytorch를 ONNX로 변환

In [ ]:
import torch
import onnx
#https://github.com/pytorch/vision/blob/main/torchvision/models/segmentation/deeplabv3.py

#용범님이 upload한 deeplayv3.py 파일이 ipynb 실행 폴더에 위치
from deeplabv3 import deeplabv3_mobilenet_v3_large

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch_model = deeplabv3_mobilenet_v3_large().to(device)
#용범님이 upload한 model_weights.pth 파일이 ipynb 실행 폴더에 위치
model_path = './model_weights.pth'
torch_model.load_state_dict(torch.load(model_path))
torch_model.eval()

batch_size = 1
#[batch_size, channels, height, width]
x = torch.randn(batch_size, 3, 480, 640).to(device)

# 모델 변환
torch.onnx.export(torch_model,               # 실행될 모델
                  x,                         # 모델 입력값 (튜플 또는 여러 입력값들도 가능)
                  "onnx.onnx",   # 모델 저장 경로 (파일 또는 파일과 유사한 객체 모두 가능)
                  #export_params=True,        # 모델 파일 안에 학습된 모델 가중치를 저장할지의 여부 default True
                  #opset_version=12,          # 모델을 변환할 때 사용할 ONNX 버전
                  #do_constant_folding=False,  # 최적하시 상수폴딩을 사용할지의 여부 Default False
                  verbose = True,
                  input_names = ['actual_input_1'],   # 모델의 입력값을 가리키는 이름
                  output_names = ['output1'] # 모델의 출력값을 가리키는 이름
                  )
onnx_model = onnx.load("onnx.onnx")
onnx.checker.check_model(onnx_model)
onnx.helper.printable_graph(onnx_model.graph)

ONNX를 TensorFlow로 변환

In [ ]:
!pip install onnx_tf
!sudo apt update --yes
!sudo apt install python3-dev python3-pip python3-venv --yes
!pip install --upgrade tensorflow

In [ ]:
from onnx_tf.backend import prepare
import onnx

TF_PATH = "./pb.pb" # where the representation of tensorflow model will be stored
ONNX_PATH = "./onnx.onnx" # path to my existing ONNX model
onnx_model = onnx.load(ONNX_PATH)  # load onnx model

# prepare function converts an ONNX model to an internel representation
# of the computational graph called TensorflowRep and returns
# the converted representation.
tf_rep = prepare(onnx_model)  # creating TensorflowRep object

# export_graph function obtains the graph proto corresponding to the ONNX
# model associated with the backend representation and serializes
# to a protobuf file.
tf_rep.export_graph(TF_PATH)

TensorFlow를 TensorFlow Lite로 변환

In [ ]:
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model("./pb.pb") # path to the SavedModel directory

#optimization
#https://github.com/sithu31296/PyTorch-ONNX-TFLite
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.target_spec.supported_ops = [
#     tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TFLite ops
#     tf.lite.OpsSet.SELECT_TF_OPS  # enable TF ops
# ]
tflite_model = converter.convert()

# Save the model.
with open('tflite.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image 
import torchvision.transforms as transforms

filename = './test_images/MP_SEL_SUR_000002.jpg'
img = Image.open(filename)
img = img.resize((640, 480))
to_tesnsor = transforms.ToTensor()
input_data = to_tesnsor(img)
input_data = input_data.unsqueeze(0)
print(input_data.size())
to_pil = transforms.ToPILImage()
img = to_pil(input_data.squeeze())
img.save('./resize.jpg')

#transfom compose 방법
# input_image = Image.open(filename)
# input_image = input_image.convert("RGB")
# preprocess = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Resize((480, 640)),
#     #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])
# input_data = preprocess(input_image)
# input_data = input_data.unsqueeze(0) # create a mini-batch as expected by the model
# print(input_data.size())


interpreter = tf.lite.Interpreter(model_path='tflite.tflite')
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
#input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
output_data = output_data.argmax(1)

import matplotlib.pyplot as plt
fig, axes = plt.subplots(1, 2, dpi=200)
axes[0].imshow(output_data[0])
axes[1].imshow(input_data[0].cpu().numpy().transpose([1, 2, 0]))